### How to: prepare a PyTorch Dataset

we show how to, starting from a csv file, we wrap it in a PyTorch Dataset

In [1]:
# using Pandas for Data Import from csv
import pandas as pd

from sklearn.preprocessing import LabelEncoder

import torch
from torch.utils.data import Dataset

In [2]:
# globals
DEBUG = False

In [17]:
class AttritionDataset(Dataset):
    """Oracle Attrition dataset."""

    # cat cols are identified dynamically
    # the rule used: each cols with < 10 distinct values
    def identify_categoricals(self):
        # theshol to identify categoricals cols
        THR = 10

        nunique = self.df.nunique()
        types = self.df.dtypes

        self.categorical_columns = []

        for col in self.df.columns:
            # identifichiamo come categoriche tutte le colonne che soddisfano questa condizione
            # la soglia la possiamo cambiare
            if types[col] == "object" or nunique[col] < THR:
                if DEBUG:
                    print(f"{col} distinct values: {self.df[col].nunique()}")

                self.categorical_columns.append(col)

    def codify_categoricals(self):
        for col in self.categorical_columns:
            # codifichiamo i categorici con LabelEncoder
            l_enc = LabelEncoder()
            self.df[col] = l_enc.fit_transform(self.df[col].values)

    """
    In __init__ we encapsulate the logic for loading the data from csv
    remove unneeded cols
    identify categoricals
    codify as integer, using LabelEncoder categoricals
    """

    def __init__(self, csv_file):
        """Initializes instance of class AttritionDataset.

        Args:
            csv_file (str): Path to the csv file with the data.

        """
        # here we read the entire csv
        self.df = pd.read_csv(csv_file)

        # cols not to be used
        self.cols_to_drop = [
            "Directs",
            "name",
            "Over18",
            "WeeklyWorkedHours",
            "EmployeeNumber",
        ]

        self.target = "Attrition"

        # dropping cols not to be used
        self.df = self.df.drop(columns=self.cols_to_drop)

        # label encoding of categoricals
        self.identify_categoricals()
        self.codify_categoricals()

        # Save features and target as tensors
        self.X = torch.from_numpy(self.df.drop(self.target, axis=1).values)
        self.y = torch.from_numpy(self.df[self.target].values)

    """
    A PyTorch Dataset has two fundamentals methods
    __len__ must return the number of records in the dataset
    __get_item must return item of index idx
    """

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        # Convert idx from tensor to list due to pandas bug (that arises when using pytorch's random_split)
        # if isinstance(idx, torch.Tensor):
        #    idx = idx.tolist()

        return (self.X[idx], self.y[idx])

#### Instantiate the dataset from csv file

In [41]:
attrition_path = "/opt/notebooks/ads-examples/oracle_data/orcl_attrition.csv"

attrition_ds = AttritionDataset(attrition_path)

#### Get # of records and one item from the dataset

In [42]:
INDEX = 10
print(f"The number of records in the dataset is: {len(attrition_ds)}")
print()
print(f"One example is: {attrition_ds[INDEX]}")

The number of records in the dataset is: 1470

One example is: (tensor([   36,     0,  3710,     2,    17,     2,     3,     0,     1,    84,
            3,     0,     2,     1,     1,  2426, 16479,     0,     0,    13,
            0,     2,     1,     6,     5,     2,     5,     4,     0,     3]), tensor(0))


In [43]:
# we can separate vec and label this way
vec, label = attrition_ds[INDEX]

In [44]:
label

tensor(0)

In [45]:
vec.shape

torch.Size([30])